In [56]:
from music21 import converter, instrument, note, chord, key, tempo, duration, stream

import numpy as np
import pandas as pd

from os import listdir, path
from sys import maxsize, getsizeof

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from keras.models import Sequential, load_model
from keras.layers import LSTM, Dropout, Dense, Activation
from keras.callbacks import ModelCheckpoint


In [2]:
#defining function to read MIDI files
def read_midi(file, time_tol = 1.e-3):
    print("Loading Music File:",file)
    notes_by_part = []
    # parsing a midi file
    midi = converter.parse(file)
    # grouping based on different instruments
    parts = instrument.partitionByInstrument(midi)
    keys_by_song = []
    notes_by_part = {}
    first_ten_by_part = {}
    for part in parts:
        notes_to_parse = part.recurse()
        notes = []
        keys = []
        bpm = None        # Beats per minute, this changes a lot throughout the song.
                          # will use to scale durations.
        offset = 0
        last_offset = 0
        all_notes_ato = []  # All notes At This Offset. Tracks the notes that need to
                            # be added in between two offsets
        for element in notes_to_parse:
          #  print(type(element), element)
            if (isinstance(element, instrument.Piano) or isinstance(element, instrument.Instrument)):
                continue
            if (isinstance(element, tempo.MetronomeMark)):    # Update bpm
                    bpm = element.number
            elif (isinstance(element, key.Key)):    # Key
                keys.append(str(element)) 
            elif (isinstance(element, note.Rest)):
                continue
            else:
                if (bpm is None):
                    print('bpm is None before first note, skipping part')
                    #break
                elif (isinstance(element, note.Rest)):
                    all_notes_ato.clear()
                    notes.append(('rest', element.duration.quarterLength))
                    last_offset = element.offset
                if (element.offset == last_offset):
                    if (isinstance(element, note.Note)):     # Note
                        all_notes_ato.append((str(element.pitch), \
                                                         element.duration.quarterLength))
                    elif (isinstance(element, chord.Chord)):
                        all_notes_ato.append(('.'.join(str(n) for n in element.pitches), \
                                                           element.duration.quarterLength))                                   
                else: 
                    offset = element.offset
                    cur_offset = last_offset
                    if (all_notes_ato):
                        all_notes_ato.sort(key = lambda x: x[1])
                        while(cur_offset < offset):
                            shortest_duration = all_notes_ato[0][1]
                            if (shortest_duration < (offset - cur_offset)):  # write some intermediate
                                                                               # lines, for those notes
                                                                               # whose durations fall in
                                                                               # this offset interval
                                notes.append(('.'.join(n[0] for n in all_notes_ato), \
                                            all_notes_ato[0][1]))
                                cur_offset += shortest_duration
                                while(all_notes_ato and all_notes_ato[0][1] == shortest_duration):
                                    all_notes_ato.pop(0)
                                if (not all_notes_ato):
                                    notes.append(('rest', offset - cur_offset))
                                    cur_offset = offset
                                    break
                            elif (all_notes_ato[0][1] > ((offset - cur_offset) + time_tol)):  
                                # All notes leftover should be transferred, but with 
                                # their durations shortened.
                                # Added tolerance because of rounding errors.
                                corrected = []
                                for i in range(len(all_notes_ato)):
                                    corrected.append((all_notes_ato[i][0], all_notes_ato[i][1] \
                                                    - (offset - cur_offset)))
                                all_notes_ato = corrected
                                cur_offset = offset
                            else:  # they are equal (or close enough!)
                                #print('all_notes_ato[0][1], offset - cur_offset = ', \
                                #     all_notes_ato[0][1], offset - cur_offset)
                                cur_offset = offset
                                notes.append(('.'.join(n[0] for n in all_notes_ato), \
                                                all_notes_ato[0][1]))
                                all_notes_ato.clear()  # get ready for next offset interval
                    #all_notes_ato.clear()
                    if (isinstance(element, note.Note)):     # Note
                        all_notes_ato.append((str(element.pitch), element.duration.quarterLength))
                    elif (isinstance(element, chord.Chord)):
                        all_notes_ato.append(('.'.join(str(n) for n in element.pitches), \
                                                           element.duration.quarterLength))
                    last_offset = element.offset
                    
                                    # multiply the duration in units of qtr notes (beats) by 
                                    # seconds per/qtr note (60 / bpm) to get duration in seconds
                            
#                elif (isinstance(element, note.Rest)):   # Rest
#                     if (bpm is None):
#                         break
#                     notes.append(('rest', element.duration.quarterLength, element.offset))
# The rests have pretty whack quarterLengths. I'll query offset, instead to see when it's
# time to advance

        for flat_notes in midi.flat.notes:
                #print(flat_notes.offset)
                pass
        notes_by_part[part.partName] = notes
        keys_by_song.append(list(set(keys)))
    
    return keys_by_song, notes_by_part


In [3]:
directory = '/Users/gilmer/Desktop/Springboard/music_generation/composers'
sub_dir = directory + '/chopin'
songs = []
for filename in listdir(sub_dir):
    file = path.join(sub_dir, filename)
    songs.append(read_midi(file))

Loading Music File: /Users/gilmer/Desktop/Springboard/music_generation/composers/chopin/chpn_op23.mid
Loading Music File: /Users/gilmer/Desktop/Springboard/music_generation/composers/chopin/chpn-p19.mid
Loading Music File: /Users/gilmer/Desktop/Springboard/music_generation/composers/chopin/chpn_op7_2.mid
Loading Music File: /Users/gilmer/Desktop/Springboard/music_generation/composers/chopin/chpn-p18.mid
Loading Music File: /Users/gilmer/Desktop/Springboard/music_generation/composers/chopin/chpn-p24.mid
Loading Music File: /Users/gilmer/Desktop/Springboard/music_generation/composers/chopin/chpn_op7_1.mid
Loading Music File: /Users/gilmer/Desktop/Springboard/music_generation/composers/chopin/chpn-p23.mid
Loading Music File: /Users/gilmer/Desktop/Springboard/music_generation/composers/chopin/chpn-p9.mid
Loading Music File: /Users/gilmer/Desktop/Springboard/music_generation/composers/chopin/chpn-p8.mid
Loading Music File: /Users/gilmer/Desktop/Springboard/music_generation/composers/chopin/

bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is Non

bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is None before first note, skipping part
bpm is Non

In [4]:
drop_indices = []   # Drop songs where there is a key change

keys_by_song = []
notes_by_song = []
for i in range(len(songs)):
    keys, notes_by_part = songs[i]
    print(i, keys)
    new_keys = None
    for k in keys:
        if (k):
            new_keys = k
            break
    if (len(new_keys) > 1):
        drop_indices.append(i)
    else:
        keys_by_song.append(new_keys[0])
        cur_notes_by_part = []
        for notes in notes_by_part.values():
            if (notes):
                cur_notes_by_part.append(notes)   # We won't label the parts, they are
        notes_by_song.append(cur_notes_by_part)   # all piano and we will merge them
                                                  # as if they were all played on one piano
print(drop_indices)
print(keys_by_song)

0 [[], ['B- major'], []]
1 [[], ['E- major'], []]
2 [[], ['A major', 'C major'], []]
3 [[], ['A- major'], []]
4 [[], ['F major'], []]
5 [[], ['B- major'], []]
6 [[], ['F major'], []]
7 [[], ['E major'], [], []]
8 [[], ['A major'], [], []]
9 [[], ['B- major'], []]
10 [[], ['E- major'], []]
11 [[], ['B- major'], []]
12 [[], ['D- major', 'E- major'], []]
13 [[], ['D- major'], []]
14 [[], ['D major', 'B- major'], []]
15 [[], ['A major', 'D- major'], []]
16 [[], ['C major'], []]
17 [['G- major'], [], ['G- major'], []]
18 [[], ['A- major'], [], []]
19 [[], ['D major', 'B major', 'B- major'], []]
20 [[], ['A- major', 'E major'], []]
21 [[], ['D- major'], []]
22 [[], ['D- major', 'B- major'], []]
23 [[], ['A- major'], []]
24 [[], ['B major', 'F major'], []]
25 [[], ['E- major'], []]
26 [[], ['G- major'], []]
27 [[], ['C major'], []]
28 [[], ['D- major', 'E major'], []]
29 [[], ['E major'], []]
30 [[], ['E- major'], []]
31 [[], ['D major'], []]
32 [[], ['A major'], []]
33 [[], ['B major'], []]


In [5]:
# For index 17, now index 13 (after dropping those songs with multiple keys)
# i may need to merge the two different parts:
# Let's add up the durations to see if they're equal
# Wow, purty close. I wonder why. Maybe need to look at offsets
index = 13
sum_0 = 0
for i in range(len(notes_by_song[index][0])):
    sum_0 += notes_by_song[index][0][i][1]
sum_1 = 0
for i in range(len(notes_by_song[index][1])):
    sum_1 += notes_by_song[index][1][i][1]
print(sum_0, sum_1)

693.0 905.5000000000001


In [6]:
# Is this because i left out all the rests? I thought inferring the rests from offset and 
# durations would work
# Let's drop it for now and see if we have done the reading/outputting correctly by comparing
# the input and output midis. Seems to work really well!
notes_by_song.pop(13);

In [7]:
notes_by_song[0]

[[('C3.C2', 2.5),
  ('E-3.E-2', 0.5),
  ('G#3.G#2', 0.5),
  ('B-3.B-2', 0.5),
  ('C4.C3', 0.5),
  ('G#3.G#2', 0.5),
  ('E-4.E-3', 0.5),
  ('B-4.B-3', 0.5),
  ('C5.C4', 0.5),
  ('G#4.G#3', 0.5),
  ('E-5.E-4', 0.5),
  ('B-5.B-4', 0.5),
  ('C6.C5', 0.5),
  ('G5.G4', 0.5),
  ('B-5.B-4', 0.5),
  ('G#5.G#4', 0.5),
  ('G5.G4', 1.0),
  ('F#5.F#4', 0.5),
  ('rest', 1.0),
  ('F#5.F#4', 0.5),
  ('G5.G4', 0.5),
  ('F#5.F#4', 0.5),
  ('F5.F4', 0.5),
  ('F#5.F#4', 0.5),
  ('A5.A4', Fraction(1, 3)),
  ('rest', 0.0),
  ('G5.G4', Fraction(1, 3)),
  ('E-5.E-4', Fraction(1, 3)),
  ('rest', 0.0),
  ('E-5.E-4', 0.75),
  ('D5.D4', 0.25),
  ('F5.F4', Fraction(1, 3)),
  ('E-5.E-4', Fraction(1, 3)),
  ('D5.D4', Fraction(1, 3)),
  ('D5.D4', 1.0),
  ('rest', 3.0),
  ('C5.G3.E-3.C4', 1.5),
  ('G4', 0.5),
  ('D3.G3.E-4.B-4', 3.5),
  ('D2', 0.5),
  ('F#4.C4.D4', 0.5),
  ('B-4', 0.5),
  ('A4', 0.5),
  ('B-3.D4.G2.G4', 1.0),
  ('B-3.D4.G2', 1.0),
  ('D4.G4.B-3.D5', 1.0),
  ('D4.G4.B-3', 1.0),
  ('C4.E-4.G4.A3.C5', 1.

In [8]:
# When we read and include rests, we have some rests at the beginning. Let's trim all rests 
# from the beginning and ends of songs. Also, some rests of length 0. This is still a mystery.

trimmed_notes_by_song = []
for song in notes_by_song:
    i = 0
    while(song[0][i][0] == 'rest'):
        i += 1
    j = len(song[0]) - 1
    while(song[0][j][0] == 'rest'):
        j -=1
    trimmed_notes_by_song.append(song[0][i:j + 1])
trimmed_notes_by_song[0]

[('C3.C2', 2.5),
 ('E-3.E-2', 0.5),
 ('G#3.G#2', 0.5),
 ('B-3.B-2', 0.5),
 ('C4.C3', 0.5),
 ('G#3.G#2', 0.5),
 ('E-4.E-3', 0.5),
 ('B-4.B-3', 0.5),
 ('C5.C4', 0.5),
 ('G#4.G#3', 0.5),
 ('E-5.E-4', 0.5),
 ('B-5.B-4', 0.5),
 ('C6.C5', 0.5),
 ('G5.G4', 0.5),
 ('B-5.B-4', 0.5),
 ('G#5.G#4', 0.5),
 ('G5.G4', 1.0),
 ('F#5.F#4', 0.5),
 ('rest', 1.0),
 ('F#5.F#4', 0.5),
 ('G5.G4', 0.5),
 ('F#5.F#4', 0.5),
 ('F5.F4', 0.5),
 ('F#5.F#4', 0.5),
 ('A5.A4', Fraction(1, 3)),
 ('rest', 0.0),
 ('G5.G4', Fraction(1, 3)),
 ('E-5.E-4', Fraction(1, 3)),
 ('rest', 0.0),
 ('E-5.E-4', 0.75),
 ('D5.D4', 0.25),
 ('F5.F4', Fraction(1, 3)),
 ('E-5.E-4', Fraction(1, 3)),
 ('D5.D4', Fraction(1, 3)),
 ('D5.D4', 1.0),
 ('rest', 3.0),
 ('C5.G3.E-3.C4', 1.5),
 ('G4', 0.5),
 ('D3.G3.E-4.B-4', 3.5),
 ('D2', 0.5),
 ('F#4.C4.D4', 0.5),
 ('B-4', 0.5),
 ('A4', 0.5),
 ('B-3.D4.G2.G4', 1.0),
 ('B-3.D4.G2', 1.0),
 ('D4.G4.B-3.D5', 1.0),
 ('D4.G4.B-3', 1.0),
 ('C4.E-4.G4.A3.C5', 1.0),
 ('C4.E-4.G4.A3', 1.0),
 ('rest', 0.5),
 ('D

In [9]:
offset = {'A': 0, 'B': 2, 'C': 3, 'D': 5, 'E': 7, 'F': 8, 'G': 10}

def note_to_piano_idx(a_note):
    a_note, octave = a_note[:-1], int(a_note[-1])
    if (int(octave) > 8):
        print("WARNING: octave = ", octave)
        return np.array([])
    if (len(a_note) > 1):  
        if (a_note[1] == '-'): # a flat!
            return offset[a_note[0]] + 12 * (octave - 1) - 1
        elif (a_note[1] == '#'): # a sharp!
            return offset[a_note[0]] + 12 * (octave - 1) + 1
        else:
            print("Waring: note = ", a_note)
    return offset[a_note[0]] + 12 * (octave - 1)


In [10]:
# We'll transpose everything to C. The C major scale is just all the white keys
major = [1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1] # where the 0th element is a C, all notes in octave
minor = [1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0]
major = major[-3:] +  7 * major + [major[0]]  # where the 0th element is A (first key on a piano
minor = minor[-3:] +  7 * minor + [minor[0]]  # and the last element is 


def determine_key(sequence):
    # First, sum the sequence to get total play counts for each key
    keys_times_pressed = np.sum(sequence, axis = 0).astype(np.int)
    
    all_keys = ['A', 'A#', 'B', 'C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#']
    possible_keys = all_keys.copy()   # will drop elements when its no longer possible
    # for the song to be in the key    
    counts = np.zeros(12) # store the # of each key
    counts_dict = {key: 0 for key in all_keys}  # this will be valueable when a special
                                                # case is encountered
    
    for i in range(88):
        counts[i % 12] += keys_times_pressed[i]
        current_note = all_keys[i % 12]
        counts_dict[current_note] += keys_times_pressed[i]
    song_top_7_note_indices = np.argsort(counts)[-7:][::-1]
                                     # Taking the top seven, any more and we will get 
                                     # down to zero keys left in possible_keys after
                                     # 7, guaranteed.
    top_notes = [all_keys[i] for i in song_top_7_note_indices]
    for current_note in top_notes:
        last_possible_keys = possible_keys.copy()
        if (current_note == 'A'):
            for key in ['B', 'F#', 'C#', 'G#', 'D#']:
                if (key in possible_keys):
                    possible_keys.remove(key)
        elif (current_note == 'A#'):
            for key in ['A', 'C', 'D', 'E', 'G']:
                if (key in possible_keys):
                    possible_keys.remove(key)
        elif (current_note == 'B'):
            for key in ['F', 'C#', 'G#', 'D#', 'A#']:
                if (key in possible_keys):
                    possible_keys.remove(key)
        elif (current_note == 'C'):
            for key in ['D', 'A', 'E', 'B', 'F#']:
                if (key in possible_keys):
                    possible_keys.remove(key)
        elif (current_note == 'C#'):
            for key in ['C', 'G', 'F', 'D#', 'A#']:
                if (key in possible_keys):
                    possible_keys.remove(key)
        elif (current_note == 'D'):
            for key in ['E', 'B', 'F#', 'C#', 'G#']:
                if (key in possible_keys):
                    possible_keys.remove(key)
        elif (current_note == 'D#'):
            for key in ['C', 'G', 'D', 'A', 'F']:
                if (key in possible_keys):
                    possible_keys.remove(key)
        elif (current_note == 'E'):
            for key in ['F#', 'C#', 'G#', 'D#', 'A#']:
                if (key in possible_keys):
                    possible_keys.remove(key)
        elif (current_note == 'F'):
            for key in ['G', 'D', 'A', 'E', 'B', 'F#']:  # Why does this oen have six?
                if (key in possible_keys):
                    possible_keys.remove(key)
        elif (current_note == 'F#'):
            for key in ['C', 'F', 'G#', 'D#', 'A#']:
                if (key in possible_keys):
                    possible_keys.remove(key)
        elif (current_note == 'G'):
            for key in ['A', 'E', 'B', 'F#', 'C#']:
                if (key in possible_keys):
                    possible_keys.remove(key)
        elif (current_note == 'G#'):
            for key in ['C', 'G', 'D', 'F', 'A#']:
                if (key in possible_keys):
                    possible_keys.remove(key)
        if (len(possible_keys) == 0):
            if (len(last_possible_keys) == 1):
                return last_possible_keys[0]
            else:    # SPECIAL CASE: Multiple Keys are Tied
                for current_note in top_notes:
                    if (current_note in last_possible_keys):
                        relative_major = all_keys[all_keys.index(current_note) - 9] # thank you, 
                                                                              # -'ve' indexing
                        if (relative_major in last_possible_keys): 
                                                            # if the note's relative major
                                                            # is also a possible key
                            return relative_major
                        else:
                            return current_note
                            
            return last_possible_keys
        
        
    return possible_keys

In [11]:
# Alg idea:
# Begin with most common note, rule out all keys without that note
# Then add next most common note and so on. Last key to be eliminated wins
# If more than one are eliminated at once:
# pick the one whose note was played the most (unless two of the keys are 
# equivalent minor/major pairs). An example is index 2 that finishes with 
# ['A#', 'C', 'F']. F is the relative minor of C. So if F was more common,
# than C, this would be key of F minor (or for our purposes, C major)
# So even if, 'F' has more plays than C, this one should return C.
# This is a fair way to decide, what key everything is in.
# and it got the first 19/20 right!

In [12]:
n_keys_piano = 88

def transpose_sequence(sequence, transposition):
    """ Perform a right-shift on the keys' part of the vectors
      Effectively, this outputs a new sequence repesenting
      a song but transposed.
      The size of the shift is transposition"""
    if (transposition == 0):
        return sequence
    shift = transposition
    sequence, durations = sequence[:, :-1], sequence[:, -1]
    for i in range(len(sequence)):
        sequence[i] = np.concatenate((sequence[i][-shift:], \
                                      sequence[i][:-shift]))
    return np.insert(sequence, len(sequence[0]), durations, axis = 1)

def songs_to_sequences(songs, augmentation_count = 4):
    sequences = []
    indices = None
    for song in songs:
        sequence = []
        durations = []
        for element in song[0]:
            vector = np.zeros(n_keys_piano)  # The current boolean array with which keys
                                             # are being pressed. Will add an additional
                                             # 89th element for the duration (normalized
                                             # to be the fraction of this notes duration
                                             # to that of the longest in the file)
            cur_note, duration = element  # in units of quarter-notes
            if ('.' in cur_note): # chord
                notes = cur_note.split('.')
                for cur_note in notes:   
                    vector[note_to_piano_idx(cur_note)] = 1   # chords are already formatted with piano index
            elif (cur_note != 'rest'): # a note
                vector[note_to_piano_idx(cur_note)] = 1
            sequence.append(vector)
            durations.append(float(duration))
        durations /= np.max(durations)  # normalize durations and
        sequence = np.array(sequence)
        sequence = np.insert(sequence, len(sequence[0]), durations, axis = 1)
#        transpositions = np.random.permutation(11)[:augmentation_count] 
#        transpositions = np.insert(transpositions, 0, 0)
#        # 11 possible key transpositions
#        for transposition in transpositions:
#            sequences.append(transpose_sequence(sequence, transposition))
        sequences.append(sequence)        
                              
    return np.array(sequences)

In [13]:
np.set_printoptions(threshold=maxsize)
chopin_sequences = songs_to_sequences(notes_by_song)

index = 3
print(np.array(chopin_sequences)[index].sum(axis = 1, where = np.insert(np.ones(n_keys_piano), n_keys_piano, 0).astype(bool)))

[1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 1. 2. 1. 1. 2. 2. 1. 2. 2. 2. 1. 1.
 1. 2. 1. 1. 2. 3. 1. 2. 2. 2. 1. 1. 1. 2. 1. 2. 2. 2. 1. 1. 1. 2. 1. 2.
 2. 2. 1. 4. 2. 3. 3. 1. 1. 1. 2. 1. 2. 2. 3. 1. 2. 2. 2. 1. 1. 1. 2. 1.
 2. 2. 2. 3. 3. 2. 3. 3. 3. 3. 3. 2. 2. 2. 1. 1. 2. 2. 1. 2. 3. 3. 3. 3.
 3. 2. 2. 1. 1. 1. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 3. 4. 2. 4. 3. 4.
 2. 4. 4. 4. 1. 3. 1. 1. 1. 1. 1. 1. 1. 1. 1. 3. 3. 3. 3. 3. 2. 2. 1. 1.
 1. 2. 2. 1. 1. 1. 2. 3. 2. 3. 4. 2. 1. 3. 1. 1. 1. 2. 3. 5. 4. 4. 6. 5.
 3. 3. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 1.
 1. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 1. 2. 1. 2. 2. 1. 1. 1. 1. 1. 1.
 1. 1. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 1. 1. 1. 4. 2. 3. 1. 2. 1. 1.
 1. 1. 1. 1. 2. 2. 1. 2. 1. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 1. 1. 3.
 3. 2. 3. 3. 3. 3. 3. 2. 2. 1. 1. 1. 1. 2. 1. 1. 1. 2. 3. 3. 3. 3. 3. 2.
 2. 1. 1. 1. 2. 2. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 3. 4. 2. 4. 3. 4. 2. 4.
 4. 4. 1. 3. 1. 1. 1. 1. 1. 1. 1. 1. 1. 3. 3. 3. 3.

In [14]:
# Let's compare the algorithm with the keys from the file
for i in range(len(chopin_sequences)):
    print(determine_key(chopin_sequences[i]), keys_by_song[i])
    

A# B- major
['D#'] E- major
['G#'] A- major
['C'] F major
['A#'] B- major
['F'] F major
B E major
A A major
['D#'] B- major
D# E- major
C# B- major
C# D- major
C C major
['G#'] G- major
['C#'] A- major
['G#'] D- major
['D#'] A- major
F# E- major
['C'] G- major
E C major
D# E major
D E- major
['A'] D major
['B'] A major
C# B major
['C'] F# major
['D'] C major
['G'] D major
['B'] G major
C# B major
C# D- major
['C'] D- major
G# C major
['G'] A- major
['G'] G major
F# C major


In [15]:
# My key predictor doesn't quite get all of them right

# We need to compactify multiple rests occurring in sequence
compacted_sequences = []
print(len(chopin_sequences[0]))
for sequence in chopin_sequences:
    total_duration = 0
    song_sequences = []
    last_was_rest = False # bool to control the updates to total_duration
                          # and appends to song_sequences
    for vector in sequence:
        if (vector[:-1].sum() == 0): # rest
            if (last_was_rest):
                total_duration += vector[-1]
            else:
                total_duration = vector[-1]
                last_was_rest = True
        else:
            if (last_was_rest):
                if (total_duration != 0):  # to handle rests of zero duration
                    song_sequences.append(np.concatenate((np.zeros(88), [total_duration])))
                last_was_rest = False
            song_sequences.append(vector)
    compacted_sequences.append(song_sequences)
    
pd.DataFrame(compacted_sequences[3]).describe()        

2349


,0,1,2,3,4,5,6,7,8,9,...,79,80,81,82,83,84,85,86,87,88
count,753.000000,753.000000,753.000000,753.0,753.0,753.000000,753.0,753.000000,753.000000,753.0,...,753.00000,753.000000,753.0,753.0,753.0,753.0,753.0,753.0,753.0,753.000000
mean,0.096946,0.022576,0.010624,0.0,0.0,0.002656,0.0,0.001328,0.001328,0.0,...,0.00664,0.005312,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.107570
std,0.296080,0.148647,0.102593,0.0,0.0,0.051503,0.0,0.036442,0.036442,0.0,...,0.08127,0.072739,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.062717
min,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
25%,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100000
50%,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100000
75%,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100000
max,1.000000,1.000000,1.000000,0.0,0.0,1.000000,0.0,1.000000,1.000000,0.0,...,1.00000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000


In [16]:
# Okay, let's use the keys to transpose
transposed_chopin_sequences = []  # All sequences transposed to the key of C major
for i in range(len(compacted_sequences)):
    if (keys_by_song[i][0] == 'C'):
        transposed_chopin_sequences.append(chopin_sequences[i])
    else:
        notes, durations = chopin_sequences[i][:, :-1], chopin_sequences[i][:, -1]
        transposition = ord('C') - ord(keys_by_song[i][0])
        transposed_sequence = transpose_sequence(notes, transposition)
        transposed_chopin_sequences.append(np.insert(transposed_sequence,\
                                                      n_keys_piano, durations, axis = 1))
index = 3
pd.DataFrame(transposed_chopin_sequences[index]).describe()                               

,0,1,2,3,4,5,6,7,8,9,...,79,80,81,82,83,84,85,86,87,88
count,754.0,754.0,754.000000,754.0,754.000000,754.000000,754.0,754.000000,754.0,754.000000,...,754.0,754.0,754.0,754.0,754.0,754.000000,754.000000,754.000000,754.0,754.000000
mean,0.0,0.0,0.002653,0.0,0.001326,0.001326,0.0,0.021220,0.0,0.061008,...,0.0,0.0,0.0,0.0,0.0,0.096817,0.022546,0.010610,0.0,0.107692
std,0.0,0.0,0.051468,0.0,0.036418,0.036418,0.0,0.144213,0.0,0.239504,...,0.0,0.0,0.0,0.0,0.0,0.295905,0.148551,0.102525,0.0,0.062766
min,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000
25%,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.100000
50%,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.100000
75%,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.100000
max,0.0,0.0,1.000000,0.0,1.000000,1.000000,0.0,1.000000,0.0,1.000000,...,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,1.000000,0.0,1.000000


In [17]:
# Now would be a good time to convert some sequences back into MIDI format and listen to them
# We can make sure the timing is the same (relative) between the original and the new, and
# confirm that the transpose worked

def convert_to_midi(sequence, output_file = 'music.mid'):
   
    offset = 0
    output_notes = []
    
    all_notes = ['A', 'A#', 'B', 'C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#']

    # create note, chord, and rest objects
    for vector in sequence:
        converted_duration = duration.Duration()
        converted_duration.quarterLength = vector[-1]     # conveted from seconds, assuming 
                                                          #  bpm = 60 (so beats are eqaul to seconds)
        if (np.sum(vector[:-1]) > 1):  # chord
            indices_in_chord = np.argsort(vector[:-1])[-int(np.sum(vector[:-1])):]
            notes_in_chord = [all_notes[i % 12] + str((i // 12) + 1) for i in indices_in_chord]
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(current_note)
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            new_chord.duration = converted_duration
            output_notes.append(new_chord)
            
        elif (np.sum(vector[:-1]) == 1):   # note
            index = np.argmax(vector[:-1])
            new_note = all_notes[index % 12] + str((index // 12) + 1)
            new_note = note.Note(new_note)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            new_note.duration = converted_duration
            output_notes.append(new_note)
        
        elif (np.sum(vector[:-1]) == 0):   # rest
            new_rest = note.Rest()
            new_rest.offset = offset
            new_rest.duration = converted_duration
            output_notes.append(new_rest)
        offset += vector[-1]
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp = output_file)

In [ ]:
convert_to_midi(chopin_sequences[0])

In [18]:
def sequences_to_inputs(sequences, window_size = 16):
    
    X = []
    y = []
    
    for i in range(len(sequences)):
        if (len(sequences[i]) < window_size + 1):
            print("Skipping index ", i, " because the song is too short. Try a shorter window_size to include it.")
            continue
        for j in range(len(sequences[i]) - window_size - 1):
            X.append(sequences[i][j:j + window_size])
            y.append(sequences[i][j + window_size + 1])

    return np.array(X), np.array(y)

In [19]:
X, y = sequences_to_inputs(transposed_chopin_sequences)

In [24]:
convert_to_midi(X[10000], 'first_16_bars.mid')

In [29]:
X, y = sequences_to_inputs(transposed_chopin_sequences)

# let's shuffle these inputs
X, y = shuffle(X, y)

In [30]:
y.shape

(20815, 89)

In [31]:
X.shape

(20815, 16, 89)

In [32]:
getsizeof(X)

237124608

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [34]:
def lstm(n_lstm_layers = 4, n_dense_layers = 3, n_lstm_nodes = 512, dropout_rate = 0.6):
    model = Sequential()
    for i in range(n_lstm_layers - 1):
        model.add(LSTM(n_lstm_nodes, return_sequences = True))
        model.add(Dropout(dropout_rate))
    model.add(LSTM(n_lstm_nodes))
    model.add(Dropout(dropout_rate))
    model.add(Dense(n_lstm_nodes // 2))    
    model.add(Activation('relu'))
    model.add(Dropout(dropout_rate))
    for i in range(n_dense_layers - 1):
        model.add(Dense(n_lstm_nodes // 2))
        model.add(Dropout(0.6))
    model.add(Dense(89))
    model.add(Activation('sigmoid'))
    model.compile(loss = 'categorical_crossentropy', optimizer = 'RMSProp', metrics = ['accuracy'])
    return model

In [663]:
# Wow okay, that was not nearly enough Let's try with more layers, as deep as the paper
model = lstm(n_lstm_layers = 3, n_dense_layers = 2, n_lstm_nodes = 256, dropout_rate = 0.6)
mc = ModelCheckpoint('best_chopin_model_3_2_256_pt6.h5', monitor = 'val_loss', mode = 'min', save_best_only = True, verbose = 1)

In [664]:
X_train.shape

(14570, 16, 89)

In [665]:
history = model.fit(X_train, y_train, batch_size = 512, epochs = 50, \
                    validation_data = (X_test, y_test), verbose = 2, callbacks = [mc])

Epoch 1/50

Epoch 00001: val_loss improved from inf to 10.86160, saving model to best_chopin_model_3_2_256_pt6.h5
29/29 - 42s - loss: 11.5005 - accuracy: 0.0143 - val_loss: 10.8616 - val_accuracy: 0.0143
Epoch 2/50

Epoch 00002: val_loss improved from 10.86160 to 10.69299, saving model to best_chopin_model_3_2_256_pt6.h5
29/29 - 39s - loss: 11.0568 - accuracy: 0.0149 - val_loss: 10.6930 - val_accuracy: 0.0143
Epoch 3/50

Epoch 00003: val_loss improved from 10.69299 to 10.51287, saving model to best_chopin_model_3_2_256_pt6.h5
29/29 - 39s - loss: 10.8804 - accuracy: 0.0152 - val_loss: 10.5129 - val_accuracy: 0.0143
Epoch 4/50

Epoch 00004: val_loss improved from 10.51287 to 10.42811, saving model to best_chopin_model_3_2_256_pt6.h5
29/29 - 39s - loss: 10.7226 - accuracy: 0.0143 - val_loss: 10.4281 - val_accuracy: 0.0143
Epoch 5/50

Epoch 00005: val_loss improved from 10.42811 to 10.23688, saving model to best_chopin_model_3_2_256_pt6.h5
29/29 - 39s - loss: 10.5996 - accuracy: 0.0139 - v

Epoch 45/50

Epoch 00045: val_loss improved from 9.22401 to 9.20601, saving model to best_chopin_model_3_2_256_pt6.h5
29/29 - 39s - loss: 9.4394 - accuracy: 0.0137 - val_loss: 9.2060 - val_accuracy: 0.0143
Epoch 46/50

Epoch 00046: val_loss did not improve from 9.20601
29/29 - 38s - loss: 9.4392 - accuracy: 0.0137 - val_loss: 9.2130 - val_accuracy: 0.0143
Epoch 47/50

Epoch 00047: val_loss improved from 9.20601 to 9.19855, saving model to best_chopin_model_3_2_256_pt6.h5
29/29 - 40s - loss: 9.4081 - accuracy: 0.0132 - val_loss: 9.1986 - val_accuracy: 0.0143
Epoch 48/50

Epoch 00048: val_loss improved from 9.19855 to 9.19129, saving model to best_chopin_model_3_2_256_pt6.h5
29/29 - 39s - loss: 9.3969 - accuracy: 0.0136 - val_loss: 9.1913 - val_accuracy: 0.0143
Epoch 49/50

Epoch 00049: val_loss did not improve from 9.19129
29/29 - 39s - loss: 9.3823 - accuracy: 0.0137 - val_loss: 9.2139 - val_accuracy: 0.0143
Epoch 50/50

Epoch 00050: val_loss improved from 9.19129 to 9.18202, saving mo

In [666]:
history = model.fit(X_train, y_train, batch_size = 512, epochs = 50, \
                    validation_data = (X_test, y_test), verbose = 2, callbacks = [mc])

Epoch 1/50

Epoch 00001: val_loss did not improve from 9.18202
29/29 - 39s - loss: 9.3674 - accuracy: 0.0134 - val_loss: 9.1842 - val_accuracy: 0.0143
Epoch 2/50

Epoch 00002: val_loss improved from 9.18202 to 9.16509, saving model to best_chopin_model_3_2_256_pt6.h5
29/29 - 39s - loss: 9.3698 - accuracy: 0.0135 - val_loss: 9.1651 - val_accuracy: 0.0143
Epoch 3/50

Epoch 00003: val_loss did not improve from 9.16509
29/29 - 39s - loss: 9.3469 - accuracy: 0.0133 - val_loss: 9.1685 - val_accuracy: 0.0143
Epoch 4/50

Epoch 00004: val_loss improved from 9.16509 to 9.14407, saving model to best_chopin_model_3_2_256_pt6.h5
29/29 - 39s - loss: 9.3409 - accuracy: 0.0139 - val_loss: 9.1441 - val_accuracy: 0.0143
Epoch 5/50

Epoch 00005: val_loss did not improve from 9.14407
29/29 - 39s - loss: 9.3254 - accuracy: 0.0134 - val_loss: 9.1865 - val_accuracy: 0.0143
Epoch 6/50

Epoch 00006: val_loss did not improve from 9.14407
29/29 - 38s - loss: 9.3214 - accuracy: 0.0135 - val_loss: 9.1523 - val_acc

In [ ]:
# Already overfitting and accuracy basically went nowhere. Maybe let's try with to recreate
# model in paper. That didn't take that long.

In [667]:
model = lstm(n_lstm_layers = 4, n_dense_layers = 3, n_lstm_nodes = 512, dropout_rate = 0.6)
mc = ModelCheckpoint('best_chopin_model_recreate.h5', monitor = 'val_loss', mode = 'min', save_best_only = True, verbose = 1)

In [668]:
history = model.fit(X_train, y_train, batch_size = 512, epochs = 100, \
                    validation_data = (X_test, y_test), verbose = 2, callbacks = [mc])

Epoch 1/100

Epoch 00001: val_loss improved from inf to 10.93538, saving model to best_chopin_model_recreate.h5
29/29 - 174s - loss: 11.5071 - accuracy: 0.0146 - val_loss: 10.9354 - val_accuracy: 0.0143
Epoch 2/100

Epoch 00002: val_loss improved from 10.93538 to 10.81754, saving model to best_chopin_model_recreate.h5
29/29 - 172s - loss: 11.0811 - accuracy: 0.0138 - val_loss: 10.8175 - val_accuracy: 0.0143
Epoch 3/100

Epoch 00003: val_loss improved from 10.81754 to 10.62497, saving model to best_chopin_model_recreate.h5
29/29 - 170s - loss: 10.9129 - accuracy: 0.0136 - val_loss: 10.6250 - val_accuracy: 0.0157
Epoch 4/100

Epoch 00004: val_loss improved from 10.62497 to 10.50482, saving model to best_chopin_model_recreate.h5
29/29 - 173s - loss: 10.7779 - accuracy: 0.0134 - val_loss: 10.5048 - val_accuracy: 0.0143
Epoch 5/100

Epoch 00005: val_loss improved from 10.50482 to 10.36419, saving model to best_chopin_model_recreate.h5
29/29 - 204s - loss: 10.6443 - accuracy: 0.0141 - val_lo

Epoch 45/100

Epoch 00045: val_loss did not improve from 9.22130
29/29 - 170s - loss: 9.3094 - accuracy: 0.0140 - val_loss: 9.2363 - val_accuracy: 0.0147
Epoch 46/100

Epoch 00046: val_loss did not improve from 9.22130
29/29 - 169s - loss: 9.2805 - accuracy: 0.0147 - val_loss: 9.2422 - val_accuracy: 0.0146
Epoch 47/100

Epoch 00047: val_loss improved from 9.22130 to 9.17671, saving model to best_chopin_model_recreate.h5
29/29 - 171s - loss: 9.2613 - accuracy: 0.0148 - val_loss: 9.1767 - val_accuracy: 0.0143
Epoch 48/100

Epoch 00048: val_loss did not improve from 9.17671
29/29 - 169s - loss: 9.2538 - accuracy: 0.0150 - val_loss: 9.2179 - val_accuracy: 0.0143
Epoch 49/100

Epoch 00049: val_loss did not improve from 9.17671
29/29 - 170s - loss: 9.2281 - accuracy: 0.0145 - val_loss: 9.2711 - val_accuracy: 0.0143
Epoch 50/100

Epoch 00050: val_loss did not improve from 9.17671
29/29 - 170s - loss: 9.1973 - accuracy: 0.0146 - val_loss: 9.2274 - val_accuracy: 0.0143
Epoch 51/100

Epoch 00051

Epoch 96/100

Epoch 00096: val_loss did not improve from 9.06493
29/29 - 168s - loss: 8.3351 - accuracy: 0.0187 - val_loss: 9.2996 - val_accuracy: 0.0146
Epoch 97/100

Epoch 00097: val_loss did not improve from 9.06493
29/29 - 170s - loss: 8.3209 - accuracy: 0.0177 - val_loss: 9.2376 - val_accuracy: 0.0144
Epoch 98/100

Epoch 00098: val_loss did not improve from 9.06493
29/29 - 170s - loss: 8.3144 - accuracy: 0.0175 - val_loss: 9.2042 - val_accuracy: 0.0144
Epoch 99/100

Epoch 00099: val_loss did not improve from 9.06493
29/29 - 169s - loss: 8.2913 - accuracy: 0.0178 - val_loss: 9.3210 - val_accuracy: 0.0144
Epoch 100/100

Epoch 00100: val_loss did not improve from 9.06493
29/29 - 170s - loss: 8.2505 - accuracy: 0.0182 - val_loss: 9.3629 - val_accuracy: 0.0141


In [ ]:
# How is the loss decreasing while the accuracy also decreases?
# Ah because it could just be getting some of the notes right, if there are chords being played?
# And should I try categorical cross entropy for an epoch to see what the time difference is? Sure
# And also see if results are any different.

In [ ]:
# Hmm... We started overfitting, accuracy never got very high. Actually, the shallower network
# seemed to have done better.

In [26]:
model = load_model('best_chopin_model_3_2_256_pt6.h5')

In [47]:
def generate_musical_sequence(model, no_of_timesteps = 16, index = None, threshold = 0.5):
    if (index is None):
        index = np.random.randint(0, len(X_test) - 1)
        print('random index is ', index)
    random_music = X_test[index]
    original_random_music = random_music.copy()
    predictions_new = []
    for i in range(no_of_timesteps):
        random_music = random_music.reshape(1, no_of_timesteps, n_keys_piano + 1)                    
        prob = model.predict(random_music)[0]
        y_pred = [0 if p < threshold else 1 for p in prob[:-1]]
        y_pred = np.insert(y_pred, len(y_pred), prob[-1])
        # print(prob)
        predictions_new.append(y_pred)
        random_music = np.insert(random_music, len(random_music), y_pred, axis = 0)[1:, :]
    
    return original_random_music, np.array(predictions_new).astype(np.float64)

In [48]:
random_chopin_sequence, new_chopin_sequence = generate_musical_sequence(model)
convert_to_midi(random_chopin_sequence, 'random.mid')

random index is  5617


In [49]:
convert_to_midi(new_chopin_sequence, 'new.mid')

In [57]:
# Let's try a new model. Perhaps the simpler model again, with less dropout (0.4)

model = lstm(n_lstm_layers = 3, n_dense_layers = 2, n_lstm_nodes = 256, dropout_rate = 0.4)
mc = ModelCheckpoint('best_chopin_model_3_2_256_pt4.h5', monitor = 'val_loss', mode = 'min', save_best_only = True, verbose = 1)

In [58]:
history = model.fit(X_train, y_train, batch_size = 512, epochs = 50, \
                    validation_data = (X_test, y_test), verbose = 2, callbacks = [mc])

Epoch 1/50

Epoch 00001: val_loss improved from inf to 10.86028, saving model to best_chopin_model_3_2_256_pt4.h5
29/29 - 46s - loss: 11.3363 - accuracy: 0.0178 - val_loss: 10.8603 - val_accuracy: 0.0144
Epoch 2/50

Epoch 00002: val_loss improved from 10.86028 to 10.61453, saving model to best_chopin_model_3_2_256_pt4.h5
29/29 - 39s - loss: 10.9336 - accuracy: 0.0176 - val_loss: 10.6145 - val_accuracy: 0.0147
Epoch 3/50

Epoch 00003: val_loss improved from 10.61453 to 10.39017, saving model to best_chopin_model_3_2_256_pt4.h5
29/29 - 40s - loss: 10.7157 - accuracy: 0.0176 - val_loss: 10.3902 - val_accuracy: 0.0139
Epoch 4/50

Epoch 00004: val_loss improved from 10.39017 to 10.23141, saving model to best_chopin_model_3_2_256_pt4.h5
29/29 - 39s - loss: 10.5528 - accuracy: 0.0154 - val_loss: 10.2314 - val_accuracy: 0.0141
Epoch 5/50

Epoch 00005: val_loss improved from 10.23141 to 10.13727, saving model to best_chopin_model_3_2_256_pt4.h5
29/29 - 38s - loss: 10.3883 - accuracy: 0.0146 - v

Epoch 44/50

Epoch 00044: val_loss did not improve from 8.93333
29/29 - 39s - loss: 8.9547 - accuracy: 0.0137 - val_loss: 8.9486 - val_accuracy: 0.0141
Epoch 45/50

Epoch 00045: val_loss improved from 8.93333 to 8.90848, saving model to best_chopin_model_3_2_256_pt4.h5
29/29 - 39s - loss: 8.9232 - accuracy: 0.0138 - val_loss: 8.9085 - val_accuracy: 0.0141
Epoch 46/50

Epoch 00046: val_loss improved from 8.90848 to 8.89724, saving model to best_chopin_model_3_2_256_pt4.h5
29/29 - 40s - loss: 8.9171 - accuracy: 0.0137 - val_loss: 8.8972 - val_accuracy: 0.0141
Epoch 47/50

Epoch 00047: val_loss did not improve from 8.89724
29/29 - 38s - loss: 8.8915 - accuracy: 0.0136 - val_loss: 8.8981 - val_accuracy: 0.0141
Epoch 48/50

Epoch 00048: val_loss improved from 8.89724 to 8.86343, saving model to best_chopin_model_3_2_256_pt4.h5
29/29 - 38s - loss: 8.8597 - accuracy: 0.0138 - val_loss: 8.8634 - val_accuracy: 0.0141
Epoch 49/50

Epoch 00049: val_loss did not improve from 8.86343
29/29 - 39s - 

In [59]:
model = lstm(n_lstm_layers = 3, n_dense_layers = 2, n_lstm_nodes = 256, dropout_rate = 0.2)
mc = ModelCheckpoint('best_chopin_model_3_2_256_pt2.h5', monitor = 'val_loss', mode = 'min', save_best_only = True, verbose = 1)

In [60]:
history = model.fit(X_train, y_train, batch_size = 512, epochs = 50, \
                    validation_data = (X_test, y_test), verbose = 2, callbacks = [mc])

Epoch 1/50

Epoch 00001: val_loss improved from inf to 10.81960, saving model to best_chopin_model_3_2_256_pt2.h5
29/29 - 44s - loss: 11.3214 - accuracy: 0.0143 - val_loss: 10.8196 - val_accuracy: 0.0141
Epoch 2/50

Epoch 00002: val_loss improved from 10.81960 to 10.52884, saving model to best_chopin_model_3_2_256_pt2.h5
29/29 - 39s - loss: 10.8545 - accuracy: 0.0161 - val_loss: 10.5288 - val_accuracy: 0.0143
Epoch 3/50

Epoch 00003: val_loss improved from 10.52884 to 10.30948, saving model to best_chopin_model_3_2_256_pt2.h5
29/29 - 39s - loss: 10.6114 - accuracy: 0.0183 - val_loss: 10.3095 - val_accuracy: 0.0144
Epoch 4/50

Epoch 00004: val_loss improved from 10.30948 to 10.14821, saving model to best_chopin_model_3_2_256_pt2.h5
29/29 - 39s - loss: 10.4145 - accuracy: 0.0166 - val_loss: 10.1482 - val_accuracy: 0.0141
Epoch 5/50

Epoch 00005: val_loss improved from 10.14821 to 10.00068, saving model to best_chopin_model_3_2_256_pt2.h5
29/29 - 38s - loss: 10.2670 - accuracy: 0.0150 - v

Epoch 45/50

Epoch 00045: val_loss did not improve from 8.74886
29/29 - 38s - loss: 8.2941 - accuracy: 0.0140 - val_loss: 8.7682 - val_accuracy: 0.0141
Epoch 46/50

Epoch 00046: val_loss did not improve from 8.74886
29/29 - 38s - loss: 8.2668 - accuracy: 0.0143 - val_loss: 8.7680 - val_accuracy: 0.0141
Epoch 47/50

Epoch 00047: val_loss did not improve from 8.74886
29/29 - 38s - loss: 8.2319 - accuracy: 0.0141 - val_loss: 8.7835 - val_accuracy: 0.0141
Epoch 48/50

Epoch 00048: val_loss did not improve from 8.74886
29/29 - 38s - loss: 8.2137 - accuracy: 0.0145 - val_loss: 8.8042 - val_accuracy: 0.0141
Epoch 49/50

Epoch 00049: val_loss did not improve from 8.74886
29/29 - 38s - loss: 8.1615 - accuracy: 0.0142 - val_loss: 8.8171 - val_accuracy: 0.0141
Epoch 50/50

Epoch 00050: val_loss did not improve from 8.74886
29/29 - 38s - loss: 8.1484 - accuracy: 0.0138 - val_loss: 8.8198 - val_accuracy: 0.0141


In [61]:
model = lstm(n_lstm_layers = 3, n_dense_layers = 2, n_lstm_nodes = 256, dropout_rate = 0)
mc = ModelCheckpoint('best_chopin_model_3_2_256_pt0.h5', monitor = 'val_loss', mode = 'min', save_best_only = True, verbose = 1)

In [62]:
history = model.fit(X_train, y_train, batch_size = 512, epochs = 50, \
                    validation_data = (X_test, y_test), verbose = 2, callbacks = [mc])

Epoch 1/50

Epoch 00001: val_loss improved from inf to 10.69754, saving model to best_chopin_model_3_2_256_pt0.h5
29/29 - 39s - loss: 11.2667 - accuracy: 0.0189 - val_loss: 10.6975 - val_accuracy: 0.0154
Epoch 2/50

Epoch 00002: val_loss improved from 10.69754 to 10.45720, saving model to best_chopin_model_3_2_256_pt0.h5
29/29 - 38s - loss: 10.7661 - accuracy: 0.0211 - val_loss: 10.4572 - val_accuracy: 0.0146
Epoch 3/50

Epoch 00003: val_loss improved from 10.45720 to 10.22609, saving model to best_chopin_model_3_2_256_pt0.h5
29/29 - 38s - loss: 10.5181 - accuracy: 0.0199 - val_loss: 10.2261 - val_accuracy: 0.0143
Epoch 4/50

Epoch 00004: val_loss improved from 10.22609 to 10.11727, saving model to best_chopin_model_3_2_256_pt0.h5
29/29 - 38s - loss: 10.3367 - accuracy: 0.0166 - val_loss: 10.1173 - val_accuracy: 0.0143
Epoch 5/50

Epoch 00005: val_loss improved from 10.11727 to 9.95781, saving model to best_chopin_model_3_2_256_pt0.h5
29/29 - 39s - loss: 10.1960 - accuracy: 0.0149 - va

Epoch 46/50

Epoch 00046: val_loss did not improve from 8.72489
29/29 - 38s - loss: 7.3968 - accuracy: 0.0163 - val_loss: 9.0461 - val_accuracy: 0.0141
Epoch 47/50

Epoch 00047: val_loss did not improve from 8.72489
29/29 - 38s - loss: 7.3487 - accuracy: 0.0167 - val_loss: 9.1414 - val_accuracy: 0.0141
Epoch 48/50

Epoch 00048: val_loss did not improve from 8.72489
29/29 - 38s - loss: 7.2998 - accuracy: 0.0183 - val_loss: 9.2517 - val_accuracy: 0.0141
Epoch 49/50

Epoch 00049: val_loss did not improve from 8.72489
29/29 - 37s - loss: 7.2685 - accuracy: 0.0173 - val_loss: 9.1842 - val_accuracy: 0.0146
Epoch 50/50

Epoch 00050: val_loss did not improve from 8.72489
29/29 - 38s - loss: 7.2370 - accuracy: 0.0171 - val_loss: 9.1373 - val_accuracy: 0.0147


In [63]:
model = lstm(n_lstm_layers = 3, n_dense_layers = 2, n_lstm_nodes = 512, dropout_rate = 0)
mc = ModelCheckpoint('best_chopin_model_3_2_512_pt0.h5', monitor = 'val_loss', mode = 'min', save_best_only = True, verbose = 1)

In [64]:
history = model.fit(X_train, y_train, batch_size = 512, epochs = 50, \
                    validation_data = (X_test, y_test), verbose = 2, callbacks = [mc])

Epoch 1/50

Epoch 00001: val_loss improved from inf to 10.64780, saving model to best_chopin_model_3_2_512_pt0.h5
29/29 - 146s - loss: 11.2052 - accuracy: 0.0160 - val_loss: 10.6478 - val_accuracy: 0.0141
Epoch 2/50

Epoch 00002: val_loss improved from 10.64780 to 10.35096, saving model to best_chopin_model_3_2_512_pt0.h5
29/29 - 141s - loss: 10.6435 - accuracy: 0.0171 - val_loss: 10.3510 - val_accuracy: 0.0141
Epoch 3/50

Epoch 00003: val_loss improved from 10.35096 to 10.05676, saving model to best_chopin_model_3_2_512_pt0.h5
29/29 - 146s - loss: 10.3460 - accuracy: 0.0146 - val_loss: 10.0568 - val_accuracy: 0.0141
Epoch 4/50

Epoch 00004: val_loss improved from 10.05676 to 9.95502, saving model to best_chopin_model_3_2_512_pt0.h5
29/29 - 142s - loss: 10.1162 - accuracy: 0.0145 - val_loss: 9.9550 - val_accuracy: 0.0141
Epoch 5/50

Epoch 00005: val_loss improved from 9.95502 to 9.76189, saving model to best_chopin_model_3_2_512_pt0.h5
29/29 - 154s - loss: 9.9098 - accuracy: 0.0139 - v

KeyboardInterrupt: 

In [ ]:
# Wow, this is the best yet, and val_loss is still decreasing. However, the loss in training
# training is decreasing much faster. I think I need to add some more regularization. Let's go
# to 0.2